In [18]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [19]:
data = pd.read_csv('textminer/copy_text_db.csv', index_col=0)

In [20]:
data.head()

,classId,raiting,text
0,222122,1,Феерический бред со спецэфектами.
1,222122,1,Не в коем случае не смотреть в кинотеатре))) с...
2,222122,5,"Офиегнный фильм, один из лучших за послежнее в..."
3,222122,1,Ужасно. Спали полфильма.... Для галочки сходил...
4,222122,1,Тут тома и джери не хватает .


In [21]:
data = data.dropna(how='any', axis=0)

In [22]:
y = array(data['raiting'])
rawX = array(data['text'])

In [23]:
y.shape, rawX.shape

((13832,), (13832,))

In [24]:
y[y <= 3] = 0
y[y > 3] = 1

In [45]:
import re
from nltk.corpus import stopwords
import nltk
from nltk.stem.snowball import SnowballStemmer

TAG_RE = re.compile(r'<[^>]+>')
CHARACTER_RE = re.compile(r'[\r\n]')

def remove_tags(text):
    return TAG_RE.sub('', text)

def remove_symbols(text):
    try:
        return re.sub(r'[^а-яА-Яa-zA-Z]+', ' ', text).lower()  
    except TypeError as e:
        print (text)

def stemming(words):
    stemmer = SnowballStemmer("russian")
    return [stemmer.stem(word) for word in words]

def remove_stopwords(words):
    rus_stopwords = stopwords.words("russian")
    return list(filter(lambda x: x not in rus_stopwords, words)) 

def preprocessing(text):
    return " ".join(stemming(remove_tags(remove_symbols(text)).split()))

In [26]:
from stop_words import get_stop_words

In [28]:
from sklearn.cross_validation import train_test_split
rawXtrain, rawXtest, ytrain, ytest = train_test_split(rawX, y, test_size=0.25)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
vectorizer = CountVectorizer(analyzer='word', stop_words=get_stop_words('russian'))
vectorizer.fit(rawXtrain)
Xtrain = vectorizer.transform(rawXtrain)
Xtest = vectorizer.transform(rawXtest)

In [33]:
Xtrain.shape, Xtest.shape

((10374, 66989), (3458, 66989))

In [35]:
from sklearn.linear_model import LogisticRegression as logit

clf = logit().fit(Xtrain, ytrain)
predicted = clf.predict(Xtest)
predicted_proba = clf.predict_proba(Xtest)[:,1]

In [41]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import roc_auc_score

print('Accuracy', accuracy_score(ytest, predicted))
print('Precision', precision_score(ytest, predicted))
print('Recall', recall_score(ytest, predicted))
print('roc auc', roc_auc_score(ytest, predicted_proba))

Accuracy 0.816078658184
Precision 0.824248777079
Recall 0.946629213483
roc auc 0.86180664254


In [38]:
from sklearn.linear_model import LogisticRegressionCV as LRCV

clf = LRCV(Cs=logspace(-2, 4, 6), scoring='roc_auc', cv=5, n_jobs=-1).fit(Xtrain, ytrain)
predicted = clf.predict(Xtest)
predicted_proba = clf.predict_proba(Xtest)[:,1]

In [39]:
print('roc auc', roc_auc_score(ytest, predicted_proba))

roc auc 0.86180664254


In [50]:
str = np.array([preprocessing("Фильм ужасно понравился")])
str_vec = vectorizer.transform(str)
clf.predict(str_vec)

array([1])